In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 01 01:11:02 2016
@author: DIP
"""

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
import re
import os

import pandas as pd
import numpy as np
import string
from nltk.tokenize import sent_tokenize


def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^.a-zA-z0-9\s]' if remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern,'', text)
    return text



def normalize_corpus(corpus, contraction_expansion = True, text_lower_case = True, special_char_removal = True,remove_digits=True):

    normalized_corpus = []
    for doc in corpus:
        if text_lower_case:
            doc = doc.lower()

        if special_char_removal:
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits = remove_digits)
        doc = re.sub(' +',' ', doc)
        if contraction_expansion:
            doc = expand_contractions(doc)

        normalized_corpus.append(doc)
    return normalized_corpus





Based on problem proposal associated to Text summarization, various dataset was analyzed. Finally given wide range of information within new articles, CNN/Dalymail dataset was chosen. It contains various news articles ranging from political, sports, fashion, science areas. Every article contains detailed text along with few line summaries. This will be used for supervised model where algorithm will be trained and accuracies are determined. Once the model behavior is satisfactory, it will generate summary on unseen text.
The data was extracted in form of each story file through website.Once the zipped files are manually downloaded, each story file was read one after another.
Text pre processing techniques such as Replacing new line characters, and segregating text and summary was performed. As per the dataset summary was prefixed with @highlight. So regex was used to split summary and detailed text separately for every article. These are then loaded into Dataframe for further cleaning.
Pre processing steps such as lower case conversion, removal of special characters and expansion of contractions was performed. These steps are applied on entire dataframe containing text and summary segregate as 2 rows.


In [4]:
directory = r'C:\Users\sa182c\Desktop\personal\python\projects\dailymail\dailymail\articles'
t_lst = []
s_lst = []
for file in os.listdir(directory): #t_lst = []# s_lst = []
    data=open(os.path.join(directory,file), encoding = 'utf-8', errors='ignore')
    
    x = [line.strip('\n').replace("\n"," ") for line in re.split(r'@highlight', data.read() ,re.M)]
    
    summary = x[1: ]
    text = x[0]
   
    summary_str = '.'
    t_lst.append(text)
    s_lst.append(summary_str.join(summary))
    
    

news_df = pd.DataFrame(list(zip(s_lst, t_lst)), columns = ['Summary', 'Text'])


news_df2 = pd.DataFrame(list(zip(normalize_corpus(news_df['Summary']),normalize_corpus(news_df['Text']))),columns = ['clean_sum','clean_text'])
news_df.head(10)


,Summary,Text
0,Nicholas Levene must pay the nominal sum becau...,By Alex Ward A city trader who conned million...
1,Bella Rodriguez-Torres was diagnosed with stag...,"By Helen Pow PUBLISHED: 09:16 EST, 29 May 2..."
2,Jessica Weller set off on an epic adventure of...,She posed in front of the Eiffel Tower in Pari...
3,The British No 1 defeated the Canadian youngst...,Andy Murray marched into the third round of th...
4,Photographer Amos Chapple took this stunning c...,It is perceived as one of the most introverted...
5,Sir Irvine Patnick accused Liverpool fans of c...,By Lucy Osborne and Rosie Taylor PUBLISHED: ...
6,"Dubbed 'Scape', the technology can help unlock...",Scientists have developed a new microscope tha...
7,"Ryan Aprea, two, was born at just 25 weeks and...",The moment when a young boy with severe hearin...
8,Actor Stephen Collins has hit rock bottom sinc...,The family of Stephen Collins fear the 7th Hea...
9,Pochettino has left Southampton to join Totten...,Southampton midfielder Steven Davis was sad to...
